# This notebook is for getting tweets from Twitter

Importing packages


In [ ]:
!pip install tweepy
!pip install textblob
!pip install preprocessor
!pip install statistics

In [2]:
import tweepy as tw
import textblob
import preprocessor
import statistics
import numpy as np
import pandas as pd
from typing import List
import os
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

Twitter API authentication

In [3]:
api_key = "zs1VrItXow1kXKZhNtnbn8OQD"
api_secret_key = "CW47HJK9SnYjJTNVsMMdCfyjK0x3N8hKFTAXCSaTbpmqBMMMMI"
access_token = "1206982141172764672-3hFfVWIWYzz8dEuJnGnszPdhGY5FEE"
access_token_secret = "X7IXHwBdXKAix3BdmlZ9mhWSxXfB9OPFQwGstT7dSIMaa"
Bearer_token: "AAAAAAAAAAAAAAAAAAAAAMYGNwEAAAAAOcxExg4tCZGNHJ9tiUlvK3FsM58%3DOPbPEUBUDSag04nkiHSf8JWAdts7qkC34GUgeFNKrRXDk2MDMn"

auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


# Tweets query



Searching and obtaining tweets with a specific query

In [4]:
# query attributes
search_words = "#covid19 -filter:retweets"
date_since = "2021-03-01"

# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1000)

In [5]:
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)
     
print(f"new tweets retrieved: {len(tweets_copy)}")

1000it [00:16, 62.04it/s]

new tweets retrieved: 1000


Converting the dataset into CSV format

In [6]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))
    
    
tweets_df.head()

100%|██████████| 1000/1000 [12:59<00:00,  1.28it/s]


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Covid Waves - Barbara Volkwyn,South Africa,No followers requested. The volume of tweets will irritate you. Thank you. I am building up tweets for historical purposes.,2020-09-10 17:43:54,105,180,6512,False,2021-06-01 09:02:34,#SAPS Cops told they’ll face attempted murder charges should they report for duty after testing positive for Covid-19 \nhttps://t.co/lMXoOGD3Md \n#COVID19 #COVID19SA #ThirdWave,[SAPS],Twitter Web App,False
0,ET Edge,India,"ET Edge is an Economic Times Initiative, founded to empower industries & segments by sharing critical business knowledge through strategic conferences & summits",2013-10-22 11:35:04,4502,452,1894,False,2021-06-01 09:02:34,"Come, be a part of the august audience and join us at the virtual summit on 18th June 2021.\n\nBook your seat now: https://t.co/PuTFCyWhhD\n\n#SmartManufacturing #Covid19 #automation #Robotics #aitechnology #manufacturing #ai #artificialintelligence #augmentedreality #digital",None,Twitter Web App,False
0,Tag My School Magazine,Ekurhuleni,Rural and Township Schools Magazine that enhances the culture of learning by profiling it as a cool lifestyle.,2017-05-05 11:14:24,643,1272,510,False,2021-06-01 09:02:30,LGBTQI Activist on the verge of glory read more:\nhttps://t.co/8B73kKCF6S\n#NaomiOsaka #COVID19 #mentalhealth #newmonth #LGBTQ #loveislove #HappyPrideMonth2021 #HappyPride2021,"[NaomiOsaka, COVID19, mentalhealth]",Twitter for Android,False
0,Nine Dragons,Kowloon,"I am Kowloon, the Dark Side of Hong Kong. The place where you realise that Blade Runner is basically real, right here, right now.",2017-12-22 03:29:19,399,143,2077,False,2021-06-01 09:02:28,"Vaccine progress:\nLooking at the progress in the countries with the most known #COVID19 cases, plus Hong Kong &amp; China. https://t.co/pU89E5sWUj",[COVID19],Twitter Web App,False
0,Rajesh Kshitij,"Hyderabad, India",,2010-07-25 10:45:13,557,1043,5097,False,2021-06-01 09:02:25,"#HimachalPradesh - 155 New #COVID19 cases, 1878 Recoveries &amp; 16 Deaths reported in State Today till 2PM. Now Total cases in state rises to 1,90,485 including 1,75,438 Recovered,11882 Active Cases &amp; 3143 Fatalities- Health Dept\n#coronavirus @News18MP @News18CG #CoronavirusIndia https://t.co/eLVbyAJj1M","[HimachalPradesh, COVID19]",Twitter for Android,False


Clear the duplicates if there are any

In [7]:
tweets_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_df.shape}")

all tweets: (1000, 13)


Export the data

In [8]:
tweets_df.to_csv("covid19_tweets.csv", index=False)